In [15]:
import json

# as requested in comment
c = {'site': 'prpa',
     'filename': 'data/PRPA_load_cleaned_mjw.csv',
     'dir':'models',
     'features':['Load (kW)',
            'Day',
            'Weekday',
            'Hour',
            'IMF1',                                
            'IMF2',                                
            'IMF3',
            'IMF4',
            'IMF5',
            'IMF6',
            'IMF7',
            'IMF8',
            'NP1dOH'],
    'rnn_type':'gru',
    'units':24,
    'layers':2,
    'dropout':0,
    'afuncs':{'lstm':'relu','dense':'relu','gru':'relu'},
    'sequence_length':24,
    'epochs':100,
    'patience':20,
    'train_split': 0.9,
    'shift_steps': 1,
    'loss':'binary_crossentropy',
    'verbose':0,
    'output':True,
    'plots':False,
    'metrics':['accuracy']}

with open('c.json', 'w') as file:
     file.write(json.dumps(c)) # use `json.loads` to do the reverse